In [1]:
# train vgg on our dataset (in a separete notebook)

In [22]:
%matplotlib inline

import math
import os
import shutil
import string
import sys
import time

sys.path.append("../../ssd/ssd.pytorch/")

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import scipy
import scipy.misc
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# from ssd import vgg

# from importlib import reload
# reload(densenet_modified_vgg)

# %load_ext autoreload
# %autoreload 2

In [3]:
# Trainer parameters
print_freq_epochs = 5
use_cuda = True

# Dataset Parameters
batch_size = 50
# load_size = 342
# fine_size = 300
load_size = 300
fine_size = 300
c = 3
# data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])
data_mean = np.asarray([0.0, 0.0, 0.0])
num_classes = 100

# Training parameters
# architecture = 'resnet34'
# architecture = 'dense'
architecture = 'vggbn'
lr = 0.0005  # densenet default = 0.1, 
lr_init = 0.0005
momentum = 0.9 # densenet default = 0.9 
weight_decay = 1e-3 # densenet default = 1e-4
num_epochs = 45

In [4]:
# Loading data from disk
class DataLoaderDisk(object):
    def __init__(self, **kwargs):
        self.load_size = int(kwargs['load_size'])
        self.fine_size = int(kwargs['fine_size'])
        self.data_mean = np.array(kwargs['data_mean'])
        self.randomize = kwargs['randomize']
        self.data_root = os.path.join(kwargs['data_root'])

        # read data info from lists
        self.list_im = []
        self.list_lab = []
        with open(kwargs['data_list'], 'r') as f:
            for line in f:
                path, lab =line.rstrip().split(' ')
                self.list_im.append(os.path.join(self.data_root, path))
                self.list_lab.append(int(lab))
        self.list_im = np.array(self.list_im, np.object)
        self.list_lab = np.array(self.list_lab, np.int64)
        self.num = self.list_im.shape[0]
        print('# Images found:', self.num)

        # permutation
        perm = np.random.permutation(self.num) 
        self.list_im[:, ...] = self.list_im[perm, ...]
        self.list_lab[:] = self.list_lab[perm, ...]

        self._idx = 0
        
    def next_batch(self, batch_size):
        images_batch = np.zeros((batch_size, self.fine_size, self.fine_size, 3), )
        labels_batch = np.zeros(batch_size, dtype=np.double)
        for i in range(batch_size):
            image = scipy.misc.imread(self.list_im[self._idx])
            image = scipy.misc.imresize(image, (self.load_size, self.load_size))
            image = image.astype(np.float32)/255.
            image = image - self.data_mean
            if self.randomize:
                flip = np.random.random_integers(0, 1)
                if flip>0:
                    image = image[:,::-1,:]
                offset_h = np.random.random_integers(0, self.load_size-self.fine_size)
                offset_w = np.random.random_integers(0, self.load_size-self.fine_size)
            else:
                offset_h = (self.load_size-self.fine_size)//2
                offset_w = (self.load_size-self.fine_size)//2

            images_batch[i, ...] =  image[offset_h:offset_h+self.fine_size, offset_w:offset_w+self.fine_size, :]
            labels_batch[i, ...] = self.list_lab[self._idx]
            
            self._idx += 1
            if self._idx == self.num:
                self._idx = 0
        
        # Switch to NCHW ordering and convert to torch FloatTensor
        images_batch = torch.from_numpy(images_batch.swapaxes(2, 3).swapaxes(1, 2)).float()
        labels_batch = torch.from_numpy(labels_batch).long()
        return images_batch, labels_batch
    
    def size(self):
        return self.num

    def reset(self):
        self._idx = 0
        
def construct_dataloader_disk():
    # Construct DataLoader
    opt_data_train = {
        #'data_h5': 'miniplaces_128_train.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        #'data_h5': 'miniplaces_128_val.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoaderDisk(**opt_data_train)
    loader_val = DataLoaderDisk(**opt_data_val)

    return (loader_train, loader_val)

def construct_dataloader_disk_trainval():
    opt_data_trainval = {
        #'data_h5': 'miniplaces_128_val.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/trainval.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }
    loader_valtrain = DataLoaderDisk(**opt_data_trainval)

    return (loader_valtrain)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def adjust_learning_rate(lr, optimizer, epoch):
    """Calculates a learning rate of the initial LR decayed by 10 every 30 epochs"""
    lr = lr_init * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def save_checkpoint(filename, model, state, is_best, epoch):
    torch.save(state, "models/"+filename) #"densenet121__retraining.tar"
    if is_best:
        torch.save(model, "results/"+filename)

# Define and load VGG model

In [5]:
class VGG(nn.Module):
    def __init__(self, base_vgg_layers, num_classes=100):
        super(VGG, self).__init__()
        self.base_vgg = base_vgg
        self.layers = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=(3, 3), stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=3, padding=1),
        )
        self.classifier = nn.Sequential(
            nn.Linear(7 * 7 * 512, 4096),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(4096, 4096),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(4096, num_classes),
        )
        self._initialize_weights()

    def features(self, x):
        for layer in self.base_vgg:
            x = layer(x)

        for layer in self.layers:
            x = layer(x)

        return x
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.layers:
            if isinstance(m, nn.Conv2d):
                print("Initializing", m)
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()

#         m = self.classifier[0]
#         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
#         m.weight.data.normal_(0, math.sqrt(2. / n))
#         if m.bias is not None:
#             m.bias.data.zero_()
            
        for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
#                 m.weight.data.normal_(0, math.sqrt(2. / n))
#                 if m.bias is not None:
#                     m.bias.data.zero_()
#             el
            if isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

# From ssd.py
# This function is derived from torchvision VGG make_layers()
# https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
def vgg(cfg, in_channels, batch_norm=False):
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'C':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
               nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    return layers

vgg_base_configuration = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M', 512, 512, 512]

In [6]:
ssd_vgg_weights = {}
for key, weights in torch.load("../../ssd/ssd.pytorch/weights/ssd300_MiniPlaces_0.pth").items():
    # Find weights with keys prefixed by "vgg.", and remove prefix
    if key[:4] == 'vgg.':
        ssd_vgg_weights[key[4:]] = weights

base_vgg = nn.ModuleList(vgg(vgg_base_configuration, in_channels=3))
base_vgg.load_state_dict(ssd_vgg_weights)

In [7]:
model = VGG(base_vgg, num_classes=num_classes)

if use_cuda:
    model = model.cuda(device_id=0)

Initializing Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
model

VGG (
  (base_vgg): ModuleList (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pa

# Fine tune

In [25]:
# train and validate methods adapted from https://github.com/pytorch/examples/blob/master/imagenet/main.py

def train(train_loader, model, criterion, optimizer, epoch, text_file):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i in range(int(train_loader.size()/batch_size)):
        input, target = train_loader.next_batch(batch_size)
        target = target.long()
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
        target_var = target_var.long()
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
                
        if i % print_freq_epochs == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, train_loader.size()/batch_size, batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))
            
            with open(text_file, "a+") as f:
                f.write('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(
                   epoch, i, train_loader.size()/batch_size, batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))
            
    with open(text_file, "a+") as f:
        f.write(str(epoch)+str(",")+str(i)+str(",")+str(batch_time.val)+str(",")+str(data_time.val)+str(",")+str(losses.avg)+str(",")+str(top1.avg)+str(",")+str(top5.avg)+"\n")
        
def validate(val_loader, model, criterion, text_file, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

            with open(text_file, "a+") as f:
                f.write('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    
    with open(text_file, "a+") as f:
        f.write(str("val,")+str(epoch)+","+str(i)+str(",")+str(batch_time.val)+str(",")+str(losses.avg)+str(",")+str(top1.avg)+str(",")+str(top5.avg)+"\n")

    return top5.avg

In [10]:
criterion = nn.CrossEntropyLoss()

if use_cuda:
    criterion = criterion.cuda(device_id=0)

# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
optimizer = torch.optim.Adam(model.parameters())

In [11]:
train_loader, val_loader = construct_dataloader_disk()
trainval_loader = construct_dataloader_disk_trainval()

# Images found: 100000
# Images found: 10000
# Images found: 110000


In [12]:
# input, target = train_loader.next_batch(batch_size)
# plt.imshow(input[4,:,:,:].permute(1,2,0).numpy() + data_mean)
# print((
#         (
#             (input.sum(2) / 300)\
#                 .sum(2) / 300
#         ).sum(0) / 50
#     ).numpy())
# print(data_mean)

In [13]:
# input, target = train_loader.next_batch(batch_size)

# input.size(), target.size()
# if use_cuda:
#     target = target.cuda(async=True)
#     input = input.cuda(async=True)
# input_var = torch.autograd.Variable(input)
# target_var = torch.autograd.Variable(target)
# target_var = target_var.long()

# features = model.features(input_var)

In [14]:
# output = model(input_var)

In [15]:
# output.size()

In [ ]:
best_prec5 = 0.0

filename = "vgg_ssd_lr5E-4_bn"
text_file_train = "results/"+filename+".txt"
text_file_val = "results/"+filename+".txt"

In [ ]:
# evaluate on validation set
prec5 = validate(val_loader, model, criterion, text_file_val, epoch)

# remember best prec@1 and save checkpoint
is_best = prec5 > best_prec5
best_prec5 = max(prec5, best_prec5)

save_checkpoint(filename+".pt", model, {
    'epoch': epoch + 1,
    'arch': filename,
    'state_dict': model.state_dict(),
    'best_prec5': best_prec5,
    'optimizer' : optimizer.state_dict(),
}, is_best, epoch)

Test: [0/200.0]	Time 2.231 (2.231)	Loss 4.4287 (4.4287)	Prec@1 0.000 (0.000)	Prec@5 6.000 (6.000)
Test: [5/200.0]	Time 2.097 (2.116)	Loss 4.1272 (4.2741)	Prec@1 4.000 (2.000)	Prec@5 20.000 (13.667)
Test: [10/200.0]	Time 2.099 (2.108)	Loss 4.4592 (4.2618)	Prec@1 2.000 (2.364)	Prec@5 12.000 (14.545)
Test: [15/200.0]	Time 2.109 (2.107)	Loss 4.2156 (4.2657)	Prec@1 6.000 (2.625)	Prec@5 12.000 (13.875)
Test: [20/200.0]	Time 2.112 (2.107)	Loss 4.3420 (4.2718)	Prec@1 4.000 (2.667)	Prec@5 14.000 (13.810)
Test: [25/200.0]	Time 2.113 (2.108)	Loss 4.4265 (4.2687)	Prec@1 2.000 (2.615)	Prec@5 16.000 (13.846)
Test: [30/200.0]	Time 2.119 (2.109)	Loss 4.2273 (4.2776)	Prec@1 4.000 (2.774)	Prec@5 14.000 (14.000)
Test: [35/200.0]	Time 2.119 (2.111)	Loss 4.2569 (4.2808)	Prec@1 4.000 (2.944)	Prec@5 16.000 (14.056)
Test: [40/200.0]	Time 2.127 (2.112)	Loss 4.4299 (4.2849)	Prec@1 0.000 (2.976)	Prec@5 4.000 (13.902)
Test: [45/200.0]	Time 2.126 (2.114)	Loss 4.0928 (4.2822)	Prec@1 2.000 (2.870)	Prec@5 8.000 (13.6

In [ ]:
for epoch in range(1, num_epochs):
    # lr = adjust_learning_rate(lr, optimizer, epoch) # turn off for Adam
    print("learning rate:", lr)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, text_file_train)

    # evaluate on validation set
    prec5 = validate(val_loader, model, criterion, text_file_val, epoch)

    # remember best prec@1 and save checkpoint
    is_best = prec5 > best_prec5
    best_prec5 = max(prec5, best_prec5)

    save_checkpoint(filename+".pt", model, {
        'epoch': epoch + 1,
        'arch': filename,
        'state_dict': model.state_dict(),
        'best_prec5': best_prec5,
        'optimizer' : optimizer.state_dict(),
    }, is_best, epoch)

In [20]:
sys.last_value

NameError("name 'string' is not defined")

# Load model

In [ ]:
model_vgg = torch.load('results/vgg_ssd.pt')
state_latest = torch.load('models/vgg_ssd.pt')

In [ ]:
model_vgg

In [ ]:
state_latest

In [ ]:
model_dense = torch.load('models/best_dense201.pt') # @Ajay - this loads the pretrained densenet to transfer the weights

In [ ]:
model = densenet201() # @Ajay -> this loads densenet_modified_vgg model that is connected to vgg/ssd
    
if use_cuda:
    model = model.cuda()


In [ ]:
criterion = nn.CrossEntropyLoss()

if use_cuda:
    criterion = criterion.cuda()
#     model = model.cuda()

In [ ]:
for param in model.parameters():
    param.requires_grad = False # This prevents training of the densenet parameters

In [ ]:
model.classifier

In [ ]:
num_features = 4736
num_classes = 100 
model.classifier = nn.Linear(num_features, num_classes) #overwrite the last classifier layer

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)

In [ ]:
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay) # full model
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay) # last layer

# @Ajay - the second optimizer only takes in the parameters from the last layer

In [ ]:
# train_loader, val_loader = construct_dataloader_disk()
train_loader, val_loader = construct_dataloader_h5()

In [ ]:
# need to load weights separately, as the new model has been defined

In [ ]:
best_prec5 = 0.

In [ ]:
# model 16 epochs on Adam linear retraining

In [ ]:
for epoch in range(0,num_epochs):
#     lr = adjust_learning_rate(lr, optimizer, epoch) # off for Adam

    text_file = open("epoch_output_densenet201_ssd_val_fixed_3.txt", "w")
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, text_file)

    # evaluate on validation set
    prec5 = validate(val_loader, model, criterion, text_file)
    text_file.close()
    
    # remember best prec@1 and save checkpoint
    is_best = prec5 > best_prec5
    best_prec5 = max(prec5, best_prec5)
    save_checkpoint(model, {
        'epoch': epoch + 1,
        'arch': architecture,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec5,
        'optimizer' : optimizer.state_dict(),
    }, is_best, epoch)

In [ ]:
dummy_text_file = open("epoch_outputXX.txt", "w")
prec1 = validate(val_loader, model, criterion, dummy_text_file)

In [ ]:
# torch.save(model,'densenet_201_ssd_linear_retrained_2.pt')

In [ ]:
model = torch.load("densenet_201_ssd_linear_retrained.pt")

In [ ]:
best_prec5

In [ ]:
dense_model = torch.load('models/best_dense.pt')
# dense_model2 = torch.load('models/best_dense201.pt')
dense_model2 = torch.load('densenet_201_ssd_100p.pt')
resnet_model = torch.load('models/best_resnet34.pt')

In [ ]:
def validate_ensamble(val_loader, model1, model2, model3, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output1 = model1(input_var)
        output2 = model2(input_var)
        output3 = model3(input_var)
        output = output3+output1+output2
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
                
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top5.avg

In [ ]:
validate_ensamble(val_loader, dense_model, dense_model, dense_model2, criterion)

In [ ]:
def test(test_loader, model1):
    # switch to evaluate mode
    model1.eval()

    preds = []
    
    for i in range(int(test_loader.size() / batch_size)):
        input, paths = test_loader.next_batch(batch_size)
        if use_cuda:
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)

        # compute output
        output1 = model1(input_var)
        output = output1 
        preds.append((paths, output))

        print('Test: [{0}/{1}]\t'.format((i+1)*batch_size, test_loader.size()))

    return preds


In [ ]:
def construct_dataloader_test():
    # Construct DataLoader
    opt_data_test = {
        'data_root': '../../data/images/',
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
    }
    
    return DataLoader.DataLoaderDiskTest(**opt_data_test)

In [ ]:
test_loader = construct_dataloader_test()

In [ ]:
probs = test(test_loader, model)

In [ ]:
with open('_preds_fixed12.txt', 'w') as f:
    for paths, batch_probs in probs:
        batch_preds = batch_probs.data.topk(5, 1, True, True)[1]
        for path, top5 in zip(paths, batch_preds):
            path = 'test/' + os.path.basename(path)
            f.write('{} {} {} {} {} {}\n'.format(path, *top5))

In [ ]:
# probs

In [ ]:
loader_train, loader_val = construct_dataloader_h5()

In [ ]:
batch = loader_train.next_batch(batch_size)

In [ ]:
plt.imshow()

In [ ]:
import augmentations
from sklearn.preprocessing import scale
# plt.imshow(loader_train.im_set[0])
pd = augmentations.RandomBrightness()
X = loader_train.im_set[1]
# X = pd(X.astype(np.float32), None, None)[0][:,:,::-1]
# X /= (X.max())/255.
plt.imshow(X)

In [ ]:
plt.imshow(batch[0][0].permute(1,2,0).numpy()[:,:,:]+.5)